In [ ]:
import numpy as np
from datetime import datetime

Parameters

In [ ]:
# Pricing date (Strike date)
pricing_date = datetime(2025, 1, 28)

# Dividend Payment Dates
dividend_dates = [
    datetime(2025, 2, 28),
    datetime(2025, 5, 30),
    datetime(2025, 8, 29),
    datetime(2025, 11, 28)
]

TD = [(div_date - pricing_date).days / 365 for div_date in dividend_dates]

for i, interval in enumerate(TD):
    print(f"Dividend Date {dividend_dates[i].strftime('%Y-%m-%d')} -> T = {interval:.4f} years")


Dividend Date 2025-02-28 -> T = 0.0849 years
Dividend Date 2025-05-30 -> T = 0.3342 years
Dividend Date 2025-08-29 -> T = 0.5836 years
Dividend Date 2025-11-28 -> T = 0.8329 years


In [ ]:
# Observation Dates
obs_dates = [
    datetime(2025, 4, 28),
    datetime(2025, 7, 28),
    datetime(2025, 10, 28),
]

obs_times  = [(obs_date - pricing_date).days / 365 for obs_date in obs_dates]

for i, interval in enumerate(obs_times ):
    print(f"Observation Date {obs_dates[i].strftime('%Y-%m-%d')} -> T = {interval:.4f} years")


Observation Date 2025-04-28 -> T = 0.2466 years
Observation Date 2025-07-28 -> T = 0.4959 years
Observation Date 2025-10-28 -> T = 0.7479 years


In [ ]:
N = 365
S0 = 82.97
sigma = 0.45109
r = 0.0406
T =  (datetime(2026, 1, 28)-pricing_date).days/365
D = 0.01696 /4
coupon_times = [i/12 for i in range(1, 13)]

Stock Value Tree

In [ ]:
def Stock_tree(N, S0, sigma, r, T, D, TD):
    
    S = np.zeros([N+1, N+1])    
    
    Dt = T / N
    u = np.exp(r*Dt + sigma * (Dt)**0.5)
    d = np.exp(r*Dt - sigma * (Dt)**0.5)
    print("u = ", u, "d=", d)
    # print(TD)

    # FIRST LET'S BUILD A STOCK PRICE TREE WITH DIVIDENDS  
    # Here will use the explicit formula at different stanges in the tree
    
    S[0,0] = S0
    for i in range (1,N+1):
        for j in range(0, i+1):    
            S[i, j] = S0*(u**j)*(d**(i-j))
            if i*Dt > TD[0]: S[i, j] = S0*u**j*d**(i-j)*(1-D) 
            if i*Dt > TD[1]: S[i, j] = S0*u**j*d**(i-j)*(1-D)**2 
            if i*Dt > TD[2]: S[i, j] = S0*u**j*d**(i-j)*(1-D)**3 
            if i*Dt > TD[3]: S[i, j] = S0*u**j*d**(i-j)*(1-D)**4    
    return S

Pricing

In [ ]:
def price_EL_note(N, S0, sigma, r, T, D, TD, coupon_times, obs_times):
    # 1. Construct the stock price binomial tree (adjusted for dividends)
    S = Stock_tree(N, S0, sigma, r, T, D, TD)
    
    dt = T / N
    # Compute binomial tree parameters (consistent with Stock_tree function)
    u = np.exp(r*dt + sigma * np.sqrt(dt))
    d = np.exp(r*dt - sigma * np.sqrt(dt))
    q = (np.exp(r*dt) - d) / (u - d)
    
    # 2. Define note parameters
    face = 1000.0
    auto_call_barrier = 82.97       # Autocall trigger price: 82.97
    conversion_barrier = 70.52      # Conversion threshold at maturity
    coupon = face * (0.142/12)      # Monthly coupon payment, e.g., 1000*(0.142/12)
    
    # 3. Convert coupon and observation dates to binomial tree step indices
    coupon_indices = [int(np.ceil(t/dt)) for t in coupon_times]
    obs_indices = [int(np.ceil(t/dt)) for t in obs_times]
    print("Coupon payment steps:", coupon_indices)
    print("Observation steps:", obs_indices)
    
    # 4. Initialize the note value tree with dimensions (N+1) x (N+1)
    V = np.zeros((N+1, N+1))
    
    # 5. Determine note payoff at maturity (including final coupon)
    for j in range(N+1):
        if S[N, j] >= conversion_barrier:
            V[N, j] = face + coupon  # Full face value + final coupon payment
        else:
            # If stock price falls below the conversion threshold, apply conversion ratio (1000/70.52) and add final coupon
            V[N, j] = (face / conversion_barrier) * S[N, j] + coupon
    
    # 6. Perform backward induction to compute the note value at each node
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            # Compute expected discounted future value
            cont_value = np.exp(-r*dt) * (q * V[i+1, j+1] + (1-q) * V[i+1, j])
            # If current step is a coupon payment date, add coupon
            if i in coupon_indices:
                cont_value += coupon
            # If current step is an observation date, check autocall condition
            if i in obs_indices and S[i, j] >= auto_call_barrier:
                # Autocall: Immediately redeem the note and pay face value + current coupon
                auto_call_payoff = face + coupon
                V[i, j] = auto_call_payoff
            else:
                V[i, j] = cont_value
    return V[0, 0]

In [ ]:
note_price = price_EL_note(N, S0, sigma, r, T, D, TD, coupon_times, obs_times)

u =  1.0240059760645608 d= 0.9767740754256147
Coupon payment steps: [31, 61, 92, 122, 153, 183, 213, 244, 274, 305, 335, 365]
Observation steps: [90, 181, 273]


In [ ]:
print("EL Note Price =", note_price)

EL Note Price = 979.6134869654595
